# 数据合并与分组

In [1]:
import pandas as pd

## 1、数据合并

In [2]:
staff_df = pd.DataFrame([{'姓名': '张三', '部门': '研发部'},
                        {'姓名': '李四', '部门': '财务部'},
                        {'姓名': '赵六', '部门': '市场部'}])


student_df = pd.DataFrame([{'姓名': '张三', '专业': '计算机'},
                        {'姓名': '李四', '专业': '会计'},
                        {'姓名': '王五', '专业': '市场营销'}])

print(staff_df)
print()
print(student_df)

   姓名   部门
0  张三  研发部
1  李四  财务部
2  赵六  市场部

     专业  姓名
0   计算机  张三
1    会计  李四
2  市场营销  王五


### pd.merge() 通过键拼接列

https://blog.csdn.net/zutsoft/article/details/51498026

In [3]:
pd.merge(staff_df, student_df, how='outer', on='姓名')
# 或者：
# staff_df.merge(student_df, how='outer', on='姓名')

,姓名,部门,专业
0,张三,研发部,计算机
1,李四,财务部,会计
2,赵六,市场部,NaN
3,王五,NaN,市场营销


In [4]:
pd.merge(staff_df, student_df, how='inner', on='姓名')
# 或者：
# staff_df.merge(student_df, how='inner', on='姓名')

,姓名,部门,专业
0,张三,研发部,计算机
1,李四,财务部,会计


In [5]:
pd.merge(staff_df, student_df, how='left', on='姓名')
# 或者：
# staff_df.merge(student_df, how='left', on='姓名')

,姓名,部门,专业
0,张三,研发部,计算机
1,李四,财务部,会计
2,赵六,市场部,NaN


In [6]:
pd.merge(staff_df, student_df, how='right', on='姓名')
# 或者：
# staff_df.merge(student_df, how='right', on='姓名')

,姓名,部门,专业
0,张三,研发部,计算机
1,李四,财务部,会计
2,王五,NaN,市场营销


In [7]:
# 也可以按照索引进行合并
staff_df.set_index('姓名', inplace=True)
student_df.set_index('姓名', inplace=True)

print(staff_df)
print()
print(student_df)
print()

pd.merge(staff_df, student_df, how='outer', left_index=True, right_index=True)

     部门
姓名     
张三  研发部
李四  财务部
赵六  市场部

      专业
姓名      
张三   计算机
李四    会计
王五  市场营销



,部门,专业
姓名,,
张三,研发部,计算机
李四,财务部,会计
王五,NaN,市场营销
赵六,市场部,NaN


In [8]:
# 当数据中列名不同时，使用 left_on、right_on 属性
staff_df.reset_index(inplace=True)  # 重置为默认 index
student_df.reset_index(inplace=True)

staff_df.rename(columns={'姓名':'员工姓名'}, inplace=True)  # 修改 staff_df 的列名

print(staff_df)
print()
print(student_df)

pd.merge(staff_df, student_df, how='outer', left_on='员工姓名', right_on='姓名')

  员工姓名   部门
0   张三  研发部
1   李四  财务部
2   赵六  市场部

   姓名    专业
0  张三   计算机
1  李四    会计
2  王五  市场营销


,员工姓名,部门,姓名,专业
0,张三,研发部,张三,计算机
1,李四,财务部,李四,会计
2,赵六,市场部,NaN,NaN
3,NaN,NaN,王五,市场营销


In [9]:
# 如果两个数据集中包含相同列名（但不是要连接的列），merge() 会自动加后缀作为区分
staff_df['地区'] = ['天津', '北京', '南京']
student_df['地区'] = ['天津', '香港', '深圳']

print(staff_df)
print()
print(student_df)

  员工姓名   部门  地区
0   张三  研发部  天津
1   李四  财务部  北京
2   赵六  市场部  南京

   姓名    专业  地区
0  张三   计算机  天津
1  李四    会计  香港
2  王五  市场营销  深圳


In [10]:
pd.merge(staff_df, student_df, how='outer', left_on='员工姓名', right_on='姓名')

,员工姓名,部门,地区_x,姓名,专业,地区_y
0,张三,研发部,天津,张三,计算机,天津
1,李四,财务部,北京,李四,会计,香港
2,赵六,市场部,南京,NaN,NaN,NaN
3,NaN,NaN,NaN,王五,市场营销,深圳


In [11]:
# 也可以指定后缀名称
pd.merge(staff_df, student_df, how='outer', left_on='员工姓名', right_on='姓名', suffixes=('(公司)', '(家乡)'))

,员工姓名,部门,地区(公司),姓名,专业,地区(家乡)
0,张三,研发部,天津,张三,计算机,天津
1,李四,财务部,北京,李四,会计,香港
2,赵六,市场部,南京,NaN,NaN,NaN
3,NaN,NaN,NaN,王五,市场营销,深圳


In [12]:
# 指定多列进行合并，找出工作地址和家乡地址相同的人
pd.merge(staff_df, student_df, how='inner', left_on=['员工姓名', '地区'], right_on=['姓名', '地区'])

,员工姓名,部门,地区,姓名,专业
0,张三,研发部,天津,张三,计算机


In [13]:
# apply() 的使用
# 获取姓
staff_df['员工姓名'].apply(lambda x: x[0])

0    张
1    李
2    赵
Name: 员工姓名, dtype: object

In [14]:
# 获取名
staff_df['员工姓名'].apply(lambda x: x[1:])

0    三
1    四
2    六
Name: 员工姓名, dtype: object

In [15]:
# 结果合并
staff_df['姓'] = list(staff_df['员工姓名'].apply(lambda x: x[0]))
staff_df['名'] = list(staff_df['员工姓名'].apply(lambda x: x[1:]))

student_df['姓'] = list(map(lambda x: x[0], student_df['姓名']))
student_df['名'] = list(map(lambda x: x[1:], student_df['姓名']))

staff_df

,员工姓名,部门,地区,姓,名
0,张三,研发部,天津,张,三
1,李四,财务部,北京,李,四
2,赵六,市场部,南京,赵,六


## 2、数据分组

In [25]:
report_data = pd.read_csv('./2015.csv')
report_data.head()

,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Switzerland,Western Europe,1,7.587,0.03411,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738
1,Iceland,Western Europe,2,7.561,0.04884,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201
2,Denmark,Western Europe,3,7.527,0.03328,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204
3,Norway,Western Europe,4,7.522,0.03880,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531
4,Canada,North America,5,7.427,0.03553,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176


### groupby()

In [27]:
grouped = report_data.groupby('Region')
grouped_2 = report_data.groupby(['Country', 'Region'])  # 多层分组

print(grouped)
print()
print(type(grouped))


<class 'pandas.core.groupby.DataFrameGroupBy'>


In [28]:
# size() 查看分组后的记录数目统计结果，返回的结果是 Series 类型
grouped.size()

Region
Australia and New Zealand           2
Central and Eastern Europe         29
Eastern Asia                        6
Latin America and Caribbean        22
Middle East and Northern Africa    20
North America                       2
Southeastern Asia                   9
Southern Asia                       7
Sub-Saharan Africa                 40
Western Europe                     21
dtype: int64

In [43]:
grouped_2.size().head()

Country      Region                         
Afghanistan  Southern Asia                      1
Albania      Central and Eastern Europe         1
Algeria      Middle East and Northern Africa    1
Angola       Sub-Saharan Africa                 1
Argentina    Latin America and Caribbean        1
dtype: int64

In [45]:
# 通过索引提取各分类记录数
print(grouped.size().loc['Central and Eastern Europe'])
print()
print(grouped_2.size().loc['Afghanistan', 'Southern Asia'])

29

1


In [48]:
# get_group() 获取指定分组的结果（只能选择一个分组）
grouped.get_group('Australia and New Zealand')

,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
8,New Zealand,Australia and New Zealand,9,7.286,0.03371,1.25018,1.31967,0.90837,0.63938,0.42922,0.47501,2.26425
9,Australia,Australia and New Zealand,10,7.284,0.04083,1.33358,1.30923,0.93156,0.65124,0.35637,0.43562,2.26646


In [52]:
grouped_2.get_group(('Afghanistan', 'Southern Asia'))  # 多层分组需要传递一个元组

,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
152,Afghanistan,Southern Asia,153,3.575,0.03084,0.31982,0.30285,0.30335,0.23414,0.09719,0.3651,1.9521


In [53]:
# 对分组应用函数
grouped['Happiness Score'].mean()

Region
Australia and New Zealand          7.285000
Central and Eastern Europe         5.332931
Eastern Asia                       5.626167
Latin America and Caribbean        6.144682
Middle East and Northern Africa    5.406900
North America                      7.273000
Southeastern Asia                  5.317444
Southern Asia                      4.580857
Sub-Saharan Africa                 4.202800
Western Europe                     6.689619
Name: Happiness Score, dtype: float64

In [58]:
# 迭代 groupby 对象
for group, frame in grouped:
    mean_score = frame['Happiness Score'].mean()
    max_score = frame['Happiness Score'].max()
    min_score = frame['Happiness Score'].min()
    print('{} 平均幸福指数：{}， 最高幸福指数：{}， 最低幸福指数：{}'.format(group, mean_score, max_score, min_score))

Australia and New Zealand 平均幸福指数：7.285， 最高幸福指数：7.2860000000000005， 最低幸福指数：7.284
Central and Eastern Europe 平均幸福指数：5.332931034482758， 最高幸福指数：6.505， 最低幸福指数：4.218
Eastern Asia 平均幸福指数：5.626166666666666， 最高幸福指数：6.297999999999999， 最低幸福指数：4.874
Latin America and Caribbean 平均幸福指数：6.144681818181818， 最高幸福指数：7.226， 最低幸福指数：4.518
Middle East and Northern Africa 平均幸福指数：5.406899999999999， 最高幸福指数：7.278， 最低幸福指数：3.0060000000000002
North America 平均幸福指数：7.273， 最高幸福指数：7.4270000000000005， 最低幸福指数：7.119
Southeastern Asia 平均幸福指数：5.317444444444445， 最高幸福指数：6.797999999999999， 最低幸福指数：3.819
Southern Asia 平均幸福指数：4.580857142857143， 最高幸福指数：5.252999999999999， 最低幸福指数：3.575
Sub-Saharan Africa 平均幸福指数：4.202800000000001， 最高幸福指数：5.477， 最低幸福指数：2.839
Western Europe 平均幸福指数：6.689619047619048， 最高幸福指数：7.587000000000001， 最低幸福指数：4.857


In [61]:
# 自定义函数进行分组
# 按照幸福指数排名进行划分，1-10, 10-20, >20
# 如果自定义函数，操作针对的是index
report_data2 = report_data.set_index('Happiness Rank')

def get_rank_group(rank):
    rank_group = ''
    if rank <= 10:
        rank_group = '0 -- 10'
    elif rank <= 20:
        rank_group = '10 -- 20'
    else:
        rank_group = '> 20'
    return rank_group

grouped = report_data2.groupby(get_rank_group)
for group, frame in grouped:
    print('{} 分组的数据个数：{}'.format(group, len(frame)))

0 -- 10 分组的数据个数：10
10 -- 20 分组的数据个数：10
> 20 分组的数据个数：138


In [63]:
# 实际项目中，通常可以先人为构造出一个分组列，然后再进行groupby

# 按照score的整数部分进行分组
# 按照幸福指数排名进行划分，1-10, 10-20, >20
# 如果自定义函数，操作针对的是index
report_data['score group'] = report_data['Happiness Score'].apply(lambda score: int(score))

grouped = report_data.groupby('score group')
for group, frame in grouped:
    print('幸福指数整数部分为{}的分组数据个数：{}'.format(group, len(frame)))

幸福指数整数部分为2的分组数据个数：2
幸福指数整数部分为3的分组数据个数：19
幸福指数整数部分为4的分组数据个数：44
幸福指数整数部分为5的分组数据个数：49
幸福指数整数部分为6的分组数据个数：29
幸福指数整数部分为7的分组数据个数：15


In [64]:
import numpy as np

grouped.agg({'Happiness Score': np.mean, 'Happiness Rank': np.max})

,Happiness Score,Happiness Rank
score group,,
2,2.872000,158
3,3.706632,156
4,4.580159,137
5,5.531959,93
6,6.560379,44
7,7.356800,15


In [65]:
grouped['Happiness Score'].agg([np.mean, np.max, np.min, np.std])

,mean,amax,amin,std
score group,,,,
2,2.872000,2.905,2.839,0.046669
3,3.706632,3.995,3.006,0.248455
4,4.580159,4.971,4.033,0.253251
5,5.531959,5.995,5.007,0.329597
6,6.560379,6.983,6.003,0.290584
7,7.356800,7.587,7.119,0.146969
